In [1]:
from transformers import WhisperForConditionalGeneration
import torch
import jax.numpy as jnp
import jax
import numpy as np
jax.config.update('jax_platform_name', 'cpu')
from transformers import FlaxWhisperForConditionalGeneration

In [2]:
# load the processor and model
model_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/model_all4'
pt_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
dtype = jnp.float32
jax_model, params = FlaxWhisperForConditionalGeneration.from_pretrained(
    model_path, dtype=dtype, _do_init=False)
pt_model.config = jax_model.config

Some weights of WhisperForConditionalGeneration were not initialized from the model checkpoint at openai/whisper-small and are newly initialized: ['proj_out.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# shapes = jax.tree_map(lambda x:x.shape,params)
#shapes['model']['encoder']['conv1']#['layer_norm']#['layers']['0']
#pt_model.get_encoder().conv1.weight.shape#.layer_norm.weight.shape,pt_model.get_decoder().layer_norm.bias.shape

In [4]:
mapping = {'kernel':'weight',\
           'lm_head':'proj_out',\
           'embedding':'weight',\
           'scale':'weight'}

def set_pytorch_weights(pytorch_model, jax_dict):
    """
    Recursively set weights from the JAX dictionary to the PyTorch model.
    """
    for key, value in jax_dict.items():
        # Build the full key based on the current traversal
        key = mapping[key] if key in mapping else key
        
        # If the value is another dictionary, we recurse deeper
        if isinstance(value, dict):
            set_pytorch_weights(getattr(pytorch_model,key), value)
        else:
            # Convert JAX value to PyTorch tensor
            pytorch_tensor = torch.tensor(np.asarray(value),dtype=torch.float32)
            if isinstance(pytorch_model,(torch.nn.Linear,torch.nn.Conv1d)):
                pytorch_tensor = pytorch_tensor.T
            pytorch_tensor = torch.nn.Parameter(pytorch_tensor)
            setattr(pytorch_model, key, pytorch_tensor)

In [5]:
#%debug
set_pytorch_weights(pt_model, params)

/tmp/ipykernel_40073/566656060.py:21: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343904639/work/aten/src/ATen/native/TensorShape.cpp:3571.)
  pytorch_tensor = pytorch_tensor.T


In [6]:
pt_model.save_pretrained(model_path)

Check output matches

In [7]:
import jax.numpy as jnp
import jax

IsTPU = False
batch_size = 64
dtype = jnp.float16
speech_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/dlsprint/validation_files/'
data_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/dlsprint/validation.csv'
add_audio2 = '/home/zhenlan/Desktop/Projects/Bengali ASR/RESPIN/dev/'
num_workers = 8
output_path = ''

#from whisper_jax import FlaxWhisperForConditionalGeneration
from transformers import FlaxWhisperForConditionalGeneration
from functions import *
from functools import partial
import optax
import evaluate
from jax import random
from transformers import AutoTokenizer
import time
import warnings
warnings.filterwarnings('ignore')

In [8]:
pad_to_multiple_of = 1
max_length_gen = 24
epochs = 1
verbose = 5
learning_rate=4e-4
clip = 1e-2
# tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-large-v2", language="bn", task="transcribe")
#tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5")
tokenizer.bos_token = tokenizer.bos_token_id = None
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v2")
text = pd.read_csv(data_path)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


In [9]:
df = pd.read_csv('text',names=['code'])
df[['code','sentence']] = df["code"].str.split(" ", n=1, expand=True)
df.sentence = df.sentence.str.strip()

In [10]:
dataset1 = AudioDataset(text,speech_path,lambda x:x.path,\
                       None,orig_sr=32000, target_sr=16000)
train_loader_CE = DataLoader(dataset1, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=True,IsTPU=IsTPU,batch_size=batch_size))
train_loader_WER = DataLoader(dataset1, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=False,IsTPU=IsTPU,batch_size=batch_size))

dataset2 = AudioDataset(df,add_audio2,\
                        lambda x:x.code.split('_')[-1] + '.wav',orig_sr=16000, target_sr=16000)
train_loader_CE2 = DataLoader(dataset2, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=True,IsTPU=IsTPU,batch_size=batch_size))
train_loader_WER2 = DataLoader(dataset2, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn,tokenizer=tokenizer,feature_extractor=feature_extractor,\
                                           pad_to_multiple_of=pad_to_multiple_of,IsTrain=False,IsTPU=IsTPU,batch_size=batch_size))

In [11]:
audio,input_ids,attention_mask = next(iter(train_loader_CE))

In [12]:
# JAX
out_jax = jax_model(audio,input_ids,decoder_attention_mask=attention_mask,params=params,train=False).logits
# pytorch
pt_model.eval()
with torch.no_grad():
    out_pt = pt_model(torch.tensor(audio,dtype=torch.float32),\
                      decoder_input_ids=torch.tensor(input_ids,dtype=torch.long),\
                      decoder_attention_mask=torch.tensor(attention_mask,dtype=torch.long)).logits

In [14]:
def mean_abs(x,y,mask=None):
    if mask is None:
        return np.mean(np.abs(np.array(x)-np.array(y)))
    else:
        return np.sum(np.abs(np.array(x)-np.array(y))*mask[...,None])/np.sum(mask)/51865
mean_abs(out_jax,out_pt,attention_mask)/out_pt.abs().mean().item()

5.772960193566416e-07

In [ ]:
# encoder_jax = jax_model.encode(input_features=jnp.array(audio,dtype=jnp.float32),params=params)
# pt_model.eval()
# with torch.no_grad():
#     encoder_pt = pt_model.get_encoder()(input_features=torch.tensor(audio,dtype=torch.float32))
# mean_abs(encoder_jax.last_hidden_state,encoder_pt.last_hidden_state)
# outputs_jax = jax_model.decode(input_ids, encoder_jax,params=params)
# encoder_pt.last_hidden_state.shape
# pt_model.eval()
# with torch.no_grad():
#     outputs_pt = pt_model.get_decoder()(torch.tensor(input_ids,dtype=torch.long),encoder_hidden_states=encoder_pt.last_hidden_state)
# mean_abs(outputs_jax.logits,encoder_pt.logits)

In [ ]:
# for name, param in pt_model.named_parameters():
#     jax_name_list = name.split('.')
#     tmp_param = params
#     while jax_name_list:
#         tmp_name = jax_name_list.pop(0)
#         if tmp_name=='proj_out':
#             tmp_name = 'lm_head'
#         elif tmp_name=='weight':
#             if 'embed' in name:
#                 tmp_name = 'embedding'
#             elif 'norm' in name:
#                 tmp_name = 'scale'
#             else:
#                 tmp_name = 'kernel'
#         tmp_param = tmp_param[tmp_name]
#     pt_,jax_ = np.array(param.detach()),np.array(tmp_param)
#     if 'conv' in name or 'fc' in name or 'proj' in name:
#         jax_ = jax_.T
#     assert np.allclose(pt_,jax_ )